Downloadthe data into a `data/` folder.

In [1]:
import os, stat
import requests

import zipfile
import shutil

import docker
from docker.errors import NotFound

In [2]:
# Set files and folders.
# This is the file which will hold tutorial data.
wolfzip = 'wolfdata.zip'
# Subfolder data into which all the tutorial related materials 
# will be stored.
local_data = 'data'
# This is the local folder from where this notebook is started.
home_data = os.path.join(os.getcwd(), local_data)
# The data folder from within the docker.
docker_data = '/data'

In [3]:
# If zip file is not present, download it.
if not os.path.exists(wolfzip):
    wolfdata = requests.get('https://pythonhosted.org/OBITools/_downloads/wolf_tutorial.zip',
                    stream=True)

    with open(wolfzip, 'wb') as zippy:
        zippy.write(wolfdata.content)
        
# If data/ folder is not present, extract it from the zip archive.
if not os.path.exists(local_data):
    with zipfile.ZipFile(wolfzip, 'r') as zippy:
        zippy.extractall('.')

    # Rename extracted folder.
    os.rename('wolf_tutorial', local_data)

    os.remove(wolfzip)

    os.chmod('data', stat.S_IRWXU | stat.S_IRWXG | stat.S_IRWXO)  # https://www.tutorialspoint.com/python/os_chmod.htm

Prepare the docker container.

In [4]:
client = docker.from_env()

try:
    obt = client.containers.get('obitools')
    if obt.status == 'exited':
        print('Starting container obitools.')
        obt.start()
    else:
        print('Container obitools ready.')
except NotFound:
    print('Creating container obitools.')
    obt = client.containers.run(
        image='romunov/obitools:1.2.13',
        name='obitools',
        # This maps local data folder to /data folder in container
        volumes={home_data: {'bind': docker_data, 'mode': 'rw'}},
        tty=True, detach=True
        )

Creating container obitools.


Recover full sequence reads from forward and reverse partial reads

In [5]:
scoremin = 40
wolf_R = 'wolf_R.fastq'
wolf_F = 'wolf_F.fastq'
wolf_fastq = 'wolf.fastq'
run_ipe = f'illuminapairedend --score-min={scoremin} -r {wolf_R} {wolf_F} > {wolf_fastq}'
wolf_ipe = obt.exec_run(cmd = f'bash -c "{run_ipe}"',
                        tty=True, workdir=docker_data)

In [6]:
print(obt.exec_run(cmd=f'bash -c "head -n 4 {wolf_fastq}"', workdir=docker_data).output.decode("ascii"))  # see head of the result

@HELIUM_000100422_612GNAAXX:7:119:14871:19157#0/1_CONS ali_length=61; direction=left; seq_ab_match=47; sminR=40.0; score=115.761290673; seq_a_mismatch=7; seq_b_deletion=1; seq_b_mismatch=7; seq_a_deletion=1; score_norm=1.89772607661; seq_b_insertion=0; seq_a_insertion=0; mode=alignment; sminL=40.0; seq_a_single=46; seq_b_single=46; 
ccgcctcctttagataccccactatgcttagccctaaacacaagtaattattataacaaaatcattcgccagagtgtagcgggagtaggttaaaactcaaaggacttggcggtgctttatacccttctagaggagcctgttctaaggaggcgg
+
ddddddddddddddddddddddcddddcacdddddddddddddc\d~b~~~b~~~~~~b`ryK~|uxyXk`}~ccBccBcccBcBcccBcBccccccc~~~~b|~~xdbaddaaWcccdaaddddadacddddddcddadbbddddddddddd



Remove unaligned sequence records

In [7]:
wolf_aligned = 'wolf_aligned.fastq'
run_rsr = f'obigrep -p \\\'mode!=\"joined\"\\\' {wolf_fastq} > {wolf_aligned}'
wolf_rsr = obt.exec_run(cmd=f'bash -c "{run_rsr}"',
                       tty=True, workdir=docker_data)

In [8]:
print(wolf_rsr.output.decode('ascii'))

wolf.fastq  99.5 % |#################################################\ ] remain : 00:00:00



Assign each sequence record to the corresponding sample/marker combination

In [9]:
wolf_unid = 'unidentified.fastq'
wolf_assigned = 'wolf_aligned_assigned.fastq'
ngs_filter = 'wolf_diet_ngsfilter.txt'
run_aes = f'ngsfilter -t {ngs_filter} -u {wolf_unid} {wolf_aligned} > {wolf_assigned}'
wolf_aes = obt.exec_run(cmd=f'bash -c "{run_aes}"',
                       tty=True, workdir=docker_data)

Dereplicate reads into uniq sequences

In [10]:
wolf_unique = 'wolf_aligned_assigned_uniq.fasta'
run_drp = f'obiuniq -m sample {wolf_assigned} > {wolf_unique}'
wolf_drp = obt.exec_run(cmd=f'bash -c "{run_drp}"',
                       tty=True, workdir=docker_data)

In [11]:
run_fih = f'obiannotate -k count -k merged_sample {wolf_unique} > $$ ; mv $$ {wolf_unique}'
wolf_fih = obt.exec_run(cmd=f'bash -c "{run_fih}"',
                       tty=True, workdir=docker_data)

Denoise the data

In [12]:
wolf_unique_filtered = 'wolf_ali_assigned_uniq_c10_l80.fasta'
run_dns = f'obigrep -l 80 -p \'count>=10\' {wolf_unique} > {wolf_unique_filtered}'
wolf_dns = obt.exec_run(cmd=f'bash -c "{run_dns}"',
                       tty=True, workdir=docker_data)

Clean the sequences for PCR/sequencing errors (sequence variants)

In [13]:
wolf_unique_filtered_clean = 'wolf_ali_assigned_uniq_c10_l80_clean.fasta'
run_cpcr = f'obiclean -s merged_sample -r 0.05 -H {wolf_unique_filtered} > {wolf_unique_filtered_clean}'
wolf_cpcr = obt.exec_run(cmd=f'bash -c "{run_cpcr}"',
                       tty=True, workdir=docker_data)

In [14]:
print(obt.exec_run(cmd=f'bash -c "head -n 2 {wolf_unique_filtered_clean}"', workdir=docker_data).output.decode("ascii")) 

>HELIUM_000100422_612GNAAXX:7:22:8540:14708#0/1_CONS_SUB_SUB_CMP merged_sample={'29a_F260619': 4697, '15a_F730814': 7638}; obiclean_count={'29a_F260619': 5438, '15a_F730814': 8642}; obiclean_head=True; obiclean_cluster={'29a_F260619': 'HELIUM_000100422_612GNAAXX:7:22:8540:14708#0/1_CONS_SUB_SUB_CMP', '15a_F730814': 'HELIUM_000100422_612GNAAXX:7:22:8540:14708#0/1_CONS_SUB_SUB_CMP'}; count=12335; obiclean_internalcount=0; obiclean_status={'29a_F260619': 'h', '15a_F730814': 'h'}; obiclean_samplecount=2; obiclean_headcount=2; obiclean_singletoncount=0; 
ttagccctaaacacaagtaattaatataacaaaattattcgccagagtactaccggcaat



Assign each sequence to a taxon

In [15]:
embl = 'embl_r117'
db = 'db_v05_r117.fasta'
wolf_tagged = 'wolf_ali_assigned_uniq_c10_l80_clean_tag.fasta'
run_ass = f'ecotag -d {embl} -R {db} {wolf_unique_filtered_clean} > {wolf_tagged}'
wolf_ass = obt.exec_run(cmd=f'bash -c "{run_ass}"',
                       tty=True, workdir=docker_data)

Annotate

In [18]:
wolf_tag_ann = 'wolf_ali_assigned_uniq_c10_l80_clean_tag_ann.fasta'
run_ann = f'obiannotate  --delete-tag=scientific_name_by_db --delete-tag=obiclean_samplecount \
  --delete-tag=obiclean_count --delete-tag=obiclean_singletoncount \
  --delete-tag=obiclean_cluster --delete-tag=obiclean_internalcount \
  --delete-tag=obiclean_head --delete-tag=taxid_by_db --delete-tag=obiclean_headcount \
  --delete-tag=id_status --delete-tag=rank_by_db --delete-tag=order_name \
  --delete-tag=order {wolf_tagged} > {wolf_tag_ann}'
wolf_ann = obt.exec_run(cmd=f'bash -c "{run_ann}"',
                       tty=True, workdir=docker_data)

Create final result (a TAB file)

In [19]:
wolf_final = 'wolf_ali_assigned_uniq_c10_l80_clean_tag_ann_sort.tab'
run_tab = f'obitab -o {wolf_tag_ann} > {wolf_final}'
wolf_tab = obt.exec_run(cmd=f'bash -c "{run_tab}"',
                       tty=True, workdir=docker_data)

In [23]:
print(obt.exec_run(cmd=f'bash -c "head {wolf_final}"', tty=True, workdir=docker_data).output.decode('ascii'))

id	definition	best_identity:db_v05_r117	best_match:db_v05_r117	count	family	family_name	genus	genus_name	match_count:db_v05_r117	sample:13a_F730603	sample:15a_F730814	sample:26a_F040644	sample:29a_F260619	obiclean_status:13a_F730603	obiclean_status:15a_F730814	obiclean_status:26a_F040644	obiclean_status:29a_F260619	rank	scientific_name	species	species_list:db_v05_r117	species_name	taxid	sequence
HELIUM_000100422_612GNAAXX:7:22:8540:14708#0/1_CONS_SUB_SUB_CMP		1.0	AJ885202	12335	9850	Cervidae	9857	Capreolus	1	0	7638	0	4697	NA	h	NA	h	species	Capreolus capreolus	9858	['Capreolus capreolus']	Capreolus capreolus	9858	ttagccctaaacacaagtaattaatataacaaaattattcgccagagtactaccggcaatagcttaaaactcaaaggacttggcggtgctttataccctt
HELIUM_000100422_612GNAAXX:7:57:18459:16145#0/1_CONS_SUB_SUB		0.979797979798	AY227529	10490	55153	Sciuridae	9992	Marmota	2	0	0	10490	0	NA	NA	h	NA	genus	Marmota	NA	['Marmota monax', 'Marmota himalayana']	NA	9992	ttagccctaaacataaacattcaataaacaagaatgttcgccagagtactactagcaacagcctgaaa